### Import packeges

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib as mpl
from matplotlib.ticker import AutoLocator, AutoMinorLocator, LogLocator
import glob
from scipy.interpolate import griddata
from pathlib import Path
import h5py
import sys
from pathlib import Path

# Where am I running?
try:
    # Normal script
    here = Path(__file__).resolve().parent
except NameError:
    # Notebook / REPL
    here = Path.cwd()

phys_const_path = (here / '..' / 'phys_const').resolve()
sys.path.append(str(phys_const_path))

nsm_plots_path = (here / '..' / 'nsm_plots').resolve()
sys.path.append(str(nsm_plots_path))

nsm_plots_postproc = (here / '..' / 'nsm_instabilities').resolve()
sys.path.append(str(nsm_plots_postproc))

import phys_const as pc
import plot_functions as pf
import functions_angular_crossings as fac

### File path

In [ ]:
##################################################################################################################
# Simulation paths

# direct = '/home/erick/gw170817_1.00ye_locsim/CFI/cell_32-48-15_dom_1-1-1_km_ncell_1-1-1_novac_nomat_nonus_cdt_10.0m_att_1e-5'
# parfile = '/plt00000_particles'
# ncell = (1,1,1) # number of cells in each direction
# domain = (1e5, 1e5, 1e5) # cm
# num_particles_per_energy_bin = 92
# num_energy_bins = 13
# direct_interpolated_data = direct

direct = '/home/erick/gw170817_1.00ye_locsim/CFI/cell_32-48-15_dom_1-1-1_km_ncell_1-1-1_novacmat_cdt_10.0m_att_1e-5'
parfile = '/plt00000_particles'
ncell = (1,1,1) # number of cells in each direction
domain = (1e5, 1e5, 1e5) # cm
num_particles_per_energy_bin = 92
num_energy_bins = 13
direct_interpolated_data = direct

# direct = '/home/erick/gw170817_1.00ye_locsim/CFI/cell_32-48-15_dom_1-1-1_km_ncell_1-1-1'
# parfile = '/plt00000_particles'
# ncell = (1,1,1) # number of cells in each direction
# domain = (1e5, 1e5, 1e5) # cm
# num_particles_per_energy_bin = 92
# num_energy_bins = 13
# direct_interpolated_data = direct

# direct = '/home/erick/gw170817_1.00ye_locsim/CFI/cell_32-48-15_dom_1-1-1_km_ncell_1-1-1_novacmat'
# parfile = '/plt00000_particles'
# ncell = (1,1,1) # number of cells in each direction
# domain = (1e5, 1e5, 1e5) # cm
# num_particles_per_energy_bin = 92
# num_energy_bins = 13
# direct_interpolated_data = direct

# direct = '/home/erick/gw170817_1.00ye_locsim/CFI/cell_32-48-15_dom_1-1-1_km_ncell_1-1-1_novacmat_cdt_10.0m'
# parfile = '/plt00000_particles'
# ncell = (1,1,1) # number of cells in each direction
# domain = (1e5, 1e5, 1e5) # cm
# num_particles_per_energy_bin = 92
# num_energy_bins = 13
# direct_interpolated_data = direct

# direct = '/home/erick/gw170817_1.00ye_locsim/CFI/cell_32-48-15_dom_1-1-1_km_ncell_1-1-1_novacmat_cdt_10.0m_forplots'
# parfile = '/plt00000_particles'
# ncell = (1,1,1) # number of cells in each direction
# domain = (1e5, 1e5, 1e5) # cm
# num_particles_per_energy_bin = 92
# num_energy_bins = 13
# direct_interpolated_data = direct

# direct = '/pscratch/sd/u/uo1999/gw170817_1.00Ye/classical_transport/378_ppEb'
# parfile = '/plt00960_particles'
# ncell = (96, 96, 64) # number of cells in each direction
# domain = (96e5, 96e5, 64e5) # cm
# num_particles_per_energy_bin = 378
# num_energy_bins = 13
# direct_interpolated_data = '/pscratch/sd/u/uo1999/gw170817_1.00Ye/interpolated_SFHo_and_NuLib_quantities'

# direct = '/pscratch/sd/u/uo1999/gw170817_1.00Ye/classical_transport/1506_ppEb'
# parfile = '/plt00384_particles'
# ncell = (96, 96, 64) # number of cells in each direction
# domain = (96e5, 96e5, 64e5) # cm
# num_particles_per_energy_bin = 1506
# num_energy_bins = 13
# direct_interpolated_data = '/pscratch/sd/u/uo1999/gw170817_1.00Ye/interpolated_SFHo_and_NuLib_quantities'

##################################################################################################################

all_par_files = sorted(glob.glob(direct + '/plt*_particles'))
plt_numbers = [f.split('/')[-1].split('plt')[1].split('_particles')[0] for f in all_par_files]
plt_numbers = sorted(plt_numbers, key=lambda x: int(x))
all_par_files = [f'plt{plt_num}_particles' for plt_num in plt_numbers]

finaldir = direct.split('/')[-1]

# Calculate the volume of a single cell and the total volume
cellvolume = (domain[0] / ncell[0]) * (domain[1] / ncell[1]) * (domain[2] / ncell[2]) # cm^3
domainvolume = cellvolume * (ncell[0] * ncell[1] * ncell[2]) # cm^3

### Define cell index to be analyzed

In [ ]:
cell_index_i = 0
cell_index_j = 0
cell_index_k = 0

### Cell to be analized

In [ ]:
# Get the cell indices
cell_file_names = glob.glob(direct + parfile + '/cell_*_*_*')
cell_file_names = [file_name.split('/')[-1] for file_name in cell_file_names]
x_cell_ind = np.array([int(file_name.split('_')[1]) for file_name in cell_file_names])
y_cell_ind = np.array([int(file_name.split('_')[2]) for file_name in cell_file_names])
z_cell_ind = np.array([int((file_name.split('_')[3]).split('.')[0]) for file_name in cell_file_names])
cell_indices = np.array(list(zip(x_cell_ind, y_cell_ind, z_cell_ind)))
print('Number of cells:', len(cell_indices))
print(f'shape of cell_indices: {cell_indices.shape}')

# Get the cell indices for ix fix but iy and iz varying all available cells
x_idx_slice = cell_index_i
mask_yz_slice = cell_indices[:,0] == x_idx_slice # fixing the x index in this value
cell_indices_yz_slice = cell_indices[mask_yz_slice]
fig, ax = plt.subplots(figsize=(12, 8))
ax.scatter(cell_indices_yz_slice[:,1], cell_indices_yz_slice[:,2], color='b')
ax.scatter(cell_index_j, cell_index_k, color='r')
ax.set_xlabel('$i_y$')
ax.set_ylabel('$i_z$')
ax.set_title(f'Cell indices for fixed $i_x=${x_idx_slice}')
ax.legend()
# ax.set_xlim([-5,100])
# ax.set_ylim([-5,70])
plt.show()
plt.close(fig)

# Get the cell indices for iy fix but ix and iz varying all available cells
y_idx_slice = cell_index_j
mask_xz_slice = cell_indices[:,1] == y_idx_slice # fixing the y index in this value
cell_indices_xz_slice = cell_indices[mask_xz_slice]
fig, ax = plt.subplots(figsize=(12, 8))
ax.scatter(cell_indices_xz_slice[:,0], cell_indices_xz_slice[:,2], color='b')
ax.scatter(cell_index_i, cell_index_k, color='r')
ax.set_xlabel('$i_x$')
ax.set_ylabel('$i_z$')
ax.set_title(f'Cell indices for fixed $i_y=${y_idx_slice}')
ax.legend()
# ax.set_xlim([-5,100])
# ax.set_ylim([-5,70])
plt.show()
plt.close(fig)

# Get the cell indices for iz fix but ix and iy varying all available cells
z_idx_slice = cell_index_k
mask_xy_slice = cell_indices[:,2] == z_idx_slice # fixing the z index in this value
cell_indices_xy_slice = cell_indices[mask_xy_slice]
fig, ax = plt.subplots(figsize=(12, 8))
ax.scatter(cell_indices_xy_slice[:,0], cell_indices_xy_slice[:,1], color='b')
ax.scatter(cell_index_i, cell_index_j, color='r')
ax.set_xlabel('$i_x$')
ax.set_ylabel('$i_y$')
ax.set_title(f'Cell indices for fixed $i_z=${z_idx_slice}')
ax.legend()
# ax.set_xlim([-5,100])
# ax.set_ylim([-5,70])
plt.show()
plt.close(fig)

# Combine all cell indices from the three slices
# cell_indices_all = np.concatenate((cell_indices_yz_slice, cell_indices_xz_slice, cell_indices_xy_slice), axis=0)
cell_indices_all = np.concatenate((cell_indices_xz_slice, cell_indices_xy_slice), axis=0)
cell_indices_all = np.unique(cell_indices_all, axis=0)

### Getting temperature, electron fraction and density

In [ ]:
rho_ye_T_h5py = h5py.File(direct+'/rho_Ye_T.hdf5', 'r')

Temperature_MeV = np.array(rho_ye_T_h5py['/T_Mev'])[cell_index_i,cell_index_j,cell_index_k]
rho_g_cm3 = np.array(rho_ye_T_h5py['/rho_g|ccm'])[cell_index_i,cell_index_j,cell_index_k]
Ye = np.array(rho_ye_T_h5py['/Ye'])[cell_index_i,cell_index_j,cell_index_k]

print(f'rho_cons = {rho_g_cm3} # g/ccm')
print(f'T_cons = {Temperature_MeV} # MeV')
print(f'Ye_cons = {Ye} # n_electron - n_positron / n_barions')

neutrons_number_density_cm3 = rho_g_cm3 * (1.0 - Ye) / pc.PhysConst.Mp
protons_number_density_cm3 = rho_g_cm3 * Ye / pc.PhysConst.Mp

rho_ye_T_h5py.close()

### Compute ELN number density

In [ ]:
energybinsMeV = np.array([
    1, 3, 5.2382, 8.0097, 11.442, 15.691, 20.953, 27.468,
    35.536, 45.525, 57.895, 73.212, 92.178,
    # 115.66, 144.74, 180.75, 225.33, 280.54
])

energybinstopMeV = np.array([
    2, 4, 6.4765, 9.543, 13.34, 18.042, 23.864, 31.073,
    39.999, 51.052, 64.738, 81.685, 102.67, 
    # 128.65, 160.83, 200.67, 250, 311.08
])

energybinsbottomMeV = np.array([
    0, 2, 4, 6.4765, 9.543, 13.34, 18.042, 23.864, 31.073,
    39.999, 51.052, 64.738, 81.685, 
    # 102.67, 128.65, 160.83, 200.67, 250
])

nu_e_opacities_inv_cm = [1.1279864349828499e-08, 4.7428025628186796e-08, 1.321984832248114e-07, 3.294158114604995e-07, 7.528396092747342e-07, 1.5644452508683583e-06, 2.9487331095815796e-06, 5.1520838790595784e-06, 8.605493196949606e-06, 1.4043114431085709e-05, 2.262451577257334e-05, 3.6135649577238473e-05, 5.731900449884377e-05, 9.036764140607461e-05, 0.00014163060110313624, 0.00022059108455224182, 0.00034117744506685334, 0.0005234543477497763]
nu_x_opacities_inv_cm = [2.174662492044255e-10, 3.0391841308327543e-10, 4.466580398128411e-10, 6.371159236091845e-10, 8.969420647822444e-10, 1.2650799494987814e-09, 1.777602214007199e-09, 2.3794705306483863e-09, 3.149801121595646e-09, 4.208531959563822e-09, 5.307878395583215e-09, 6.9122806344757855e-09, 8.655465405273294e-09, 1.0987165360395685e-08, 1.3820008452372534e-08, 1.7330779319237295e-08, 2.1665700738968832e-08, 2.7031770968769257e-08]
nubar_e_opacities_inv_cm = [1.3566096423813977e-09, 4.4755275717913815e-09, 2.1546754963019424e-08, 5.47531006400814e-08, 1.1123883054511091e-07, 2.042363205567264e-07, 3.5482012940187117e-07, 5.920655243201202e-07, 9.526436748501559e-07, 1.4809548374270669e-06, 2.2289900021493824e-06, 3.2538736229147686e-06, 4.6123319631062746e-06, 6.3540654191625565e-06, 8.519538465802936e-06, 1.1153146580678764e-05, 1.4349950240540414e-05, 1.83608347661424e-05]

nu_e_opacities_inv_cm = np.array(nu_e_opacities_inv_cm)
nu_x_opacities_inv_cm = np.array(nu_x_opacities_inv_cm)
nubar_e_opacities_inv_cm = np.array(nubar_e_opacities_inv_cm)

nu_e_opacities_inv_cm = nu_e_opacities_inv_cm[0:13]
nu_x_opacities_inv_cm = nu_x_opacities_inv_cm[0:13]
nubar_e_opacities_inv_cm = nubar_e_opacities_inv_cm[0:13]

### Define function to get number density as a function of time

In [ ]:
def get_number_density_per_energy_bins(i, j, k, particlefile):

    parcellname = '/cell_' + str(i) + '_' + str(j) + '_' + str(k) + '.h5'

    particles_h5py = h5py.File(direct+particlefile+parcellname, 'r')
    N00_Re    = np.array(particles_h5py['/N00_Re'   ]) # particles
    N00_Rebar = np.array(particles_h5py['/N00_Rebar']) # particles
    N11_Re    = np.array(particles_h5py['/N11_Re'   ]) # particles
    N11_Rebar = np.array(particles_h5py['/N11_Rebar']) # particles
    N22_Re    = np.array(particles_h5py['/N22_Re'   ]) # particles
    N22_Rebar = np.array(particles_h5py['/N22_Rebar']) # particles
    time_s    = np.array(particles_h5py['/time'     ])[0] # seconds
    pupt      = np.array(particles_h5py['/pupt'     ]) # ergs
    pupx     = np.array(particles_h5py['/pupx'     ]) / pupt # unitless
    pupy     = np.array(particles_h5py['/pupy'     ]) / pupt # unitless
    pupz     = np.array(particles_h5py['/pupz'     ]) / pupt # unitless
    energyMeV = pupt / ( 1e6 * pc.CGSUnitsConst.eV ) # MeV     
    particles_h5py.close()

    n00_Re_split_energy_bins    = np.zeros(len(energybinsbottomMeV)) # particles / cm^3
    n00_Rebar_split_energy_bins = np.zeros(len(energybinsbottomMeV)) # particles / cm^3
    n11_Re_split_energy_bins    = np.zeros(len(energybinsbottomMeV)) # particles / cm^3 
    n11_Rebar_split_energy_bins = np.zeros(len(energybinsbottomMeV)) # particles / cm^3
    n22_Re_split_energy_bins    = np.zeros(len(energybinsbottomMeV)) # particles / cm^3
    n22_Rebar_split_energy_bins = np.zeros(len(energybinsbottomMeV)) # particles / cm^3

    for t in range(num_energy_bins):
        energymask = (energyMeV >= energybinsbottomMeV[t]) & (energyMeV < energybinstopMeV[t])
        if np.sum(energymask) != num_particles_per_energy_bin:
            print(f"Warning: Energy bin {t} has {np.sum(energymask)} particles, expected {num_particles_per_energy_bin}")
        n00_Re_split_energy_bins   [t] = np.sum(N00_Re[energymask])    / cellvolume # particles / cm^3
        n00_Rebar_split_energy_bins[t] = np.sum(N00_Rebar[energymask]) / cellvolume # particles / cm^3
        n11_Re_split_energy_bins   [t] = np.sum(N11_Re[energymask])    / cellvolume # particles / cm^3
        n11_Rebar_split_energy_bins[t] = np.sum(N11_Rebar[energymask]) / cellvolume # particles / cm^3
        n22_Re_split_energy_bins   [t] = np.sum(N22_Re[energymask])    / cellvolume # particles / cm^3
        n22_Rebar_split_energy_bins[t] = np.sum(N22_Rebar[energymask]) / cellvolume # particles / cm^3

    return n00_Re_split_energy_bins, n00_Rebar_split_energy_bins, n11_Re_split_energy_bins, n11_Rebar_split_energy_bins, n22_Re_split_energy_bins, n22_Rebar_split_energy_bins, time_s

### Plot the time evolution of the neutrino energy spectrum

In [ ]:

dE  = ( energybinstopMeV    - energybinsbottomMeV    ) * ( 1e6 * pc.CGSUnitsConst.eV )    # erg
dE3 = ( energybinstopMeV**3 - energybinsbottomMeV**3 ) * ( 1e6 * pc.CGSUnitsConst.eV )**3 # erg^3
E = ( energybinsMeV ) * ( 1e6 * pc.CGSUnitsConst.eV ) # erg
dOmega = 4.0 * np.pi

phase_space_factor = dOmega * dE * E**2  / pc.PhysConst.hc**3

In [ ]:
all_f00_Re_split_energy_bins    = []
all_f00_Rebar_split_energy_bins = []
all_f11_Re_split_energy_bins    = []
all_f11_Rebar_split_energy_bins = []
all_f22_Re_split_energy_bins    = []
all_f22_Rebar_split_energy_bins = []
tiempo_s = []

for dirpar in all_par_files:
    n00_Re_split_energy_bins, n00_Rebar_split_energy_bins, n11_Re_split_energy_bins, n11_Rebar_split_energy_bins, n22_Re_split_energy_bins, n22_Rebar_split_energy_bins, t_s = get_number_density_per_energy_bins(cell_index_i, cell_index_j, cell_index_k,'/'+dirpar)
    all_f00_Re_split_energy_bins   .append(np.array(n00_Re_split_energy_bins)   / phase_space_factor)
    all_f00_Rebar_split_energy_bins.append(np.array(n00_Rebar_split_energy_bins)/ phase_space_factor)
    all_f11_Re_split_energy_bins   .append(np.array(n11_Re_split_energy_bins)   / phase_space_factor)
    all_f11_Rebar_split_energy_bins.append(np.array(n11_Rebar_split_energy_bins)/ phase_space_factor)
    all_f22_Re_split_energy_bins   .append(np.array(n22_Re_split_energy_bins)   / phase_space_factor)
    all_f22_Rebar_split_energy_bins.append(np.array(n22_Rebar_split_energy_bins)/ phase_space_factor)
    tiempo_s.append(t_s)

print('tiempo_s[-1]=',tiempo_s[-1])
all_f00_Re_split_energy_bins    = np.array(all_f00_Re_split_energy_bins)
all_f00_Rebar_split_energy_bins = np.array(all_f00_Rebar_split_energy_bins)
all_f11_Re_split_energy_bins    = np.array(all_f11_Re_split_energy_bins)
all_f11_Rebar_split_energy_bins = np.array(all_f11_Rebar_split_energy_bins)
all_f22_Re_split_energy_bins    = np.array(all_f22_Re_split_energy_bins)
all_f22_Rebar_split_energy_bins = np.array(all_f22_Rebar_split_energy_bins)
tiempo_s = np.array(tiempo_s)

print(f'energybinsMeV = {energybinsMeV}')
print(f'energybinsMeV.shape = {energybinsMeV.shape}')
print(f'all_f00_Re_split_energy_bins.shape = {all_f00_Re_split_energy_bins.shape}')
print(f'all_f00_Re_split_energy_bins[1].shape = {all_f00_Re_split_energy_bins[1].shape}')
print(f'tiempo_s.shape = {tiempo_s.shape}')

net_absorption_emission_n00_inv_s_inv_cm3 = np.zeros_like(tiempo_s)
net_absorption_emission_n00bar_inv_s_inv_cm3 = np.zeros_like(tiempo_s)

for i in range(len(tiempo_s)):
    net_absorption_emission_n00_inv_s_inv_cm3[i] =  np.sum( (nu_e_opacities_inv_cm * pc.PhysConst.c) * (all_f00_Re_split_energy_bins[i] - all_f00_Re_split_energy_bins[0]) * phase_space_factor )
    net_absorption_emission_n00bar_inv_s_inv_cm3[i] =  np.sum( (nu_e_opacities_inv_cm * pc.PhysConst.c) * (all_f00_Rebar_split_energy_bins[i] - all_f00_Rebar_split_energy_bins[0]) * phase_space_factor )

net_absorption_emission_protons_inv_s_inv_cm3 = net_absorption_emission_n00bar_inv_s_inv_cm3 -  net_absorption_emission_n00_inv_s_inv_cm3
net_absorption_emission_neutrons_inv_s_inv_cm3 = net_absorption_emission_n00_inv_s_inv_cm3 - net_absorption_emission_n00bar_inv_s_inv_cm3
delta_tiempo_s = tiempo_s[1:] - tiempo_s[:-1]
cumulative_net_absorption_emission_protons_inv_s_inv_cm3 = np.cumsum(net_absorption_emission_protons_inv_s_inv_cm3[:-1] * delta_tiempo_s)
cumulative_net_absorption_emission_neutrons_inv_s_inv_cm3 = np.cumsum(net_absorption_emission_neutrons_inv_s_inv_cm3[:-1] * delta_tiempo_s)

Ye_dot_inv_s = ( pc.PhysConst.Mp / rho_g_cm3 ) * ( net_absorption_emission_n00bar_inv_s_inv_cm3 -  net_absorption_emission_n00_inv_s_inv_cm3)
cumulative_Ye_inv_s = np.cumsum(Ye_dot_inv_s[:-1] * delta_tiempo_s)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(tiempo_s, net_absorption_emission_n00_inv_s_inv_cm3, label=r'$\dot{n}_{\nu_e}$')
ax.plot(tiempo_s, net_absorption_emission_n00bar_inv_s_inv_cm3, label=r'$\dot{n}_{\bar{\nu}_e}$')
ax.set_xlabel(r'$t\,[\mathrm{s}]$')
ax.set_ylabel(r'$\dot{n}\,[\mathrm{cm}^{-3}\mathrm{s}^{-1}]$')
ax.grid(True, which='both', axis='both', linestyle='--', alpha=0.6)
leg = ax.legend(framealpha=0.0, ncol=2, fontsize=28)
pf.apply_custom_settings(ax, leg, log_scale_y=False)
plt.savefig(f"plots/{finaldir}_n_dot.pdf", bbox_inches='tight')
plt.show()
plt.close(fig)

fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(tiempo_s, net_absorption_emission_protons_inv_s_inv_cm3, label=r'$\dot{n}_{p}$')
ax.plot(tiempo_s, net_absorption_emission_neutrons_inv_s_inv_cm3, label=r'$\dot{n}_{\nu}$')
ax.set_xlabel(r'$t\,[\mathrm{s}]$')
ax.set_ylabel(r'$\dot{n}\,[\mathrm{cm}^{-3}\mathrm{s}^{-1}]$')
ax.grid(True, which='both', axis='both', linestyle='--', alpha=0.6)
leg = ax.legend(framealpha=0.0, ncol=2, fontsize=28)
pf.apply_custom_settings(ax, leg, log_scale_y=False)
plt.savefig(f"plots/{finaldir}_n_dot.pdf", bbox_inches='tight')
plt.show()
plt.close(fig)

fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(tiempo_s[1:], cumulative_net_absorption_emission_protons_inv_s_inv_cm3, label=r'$n_{p}$')
ax.plot(tiempo_s[1:], cumulative_net_absorption_emission_neutrons_inv_s_inv_cm3, label=r'$n_{n}$')  
ax.set_xlabel(r'$t\,[\mathrm{s}]$')
ax.set_ylabel(r'$n - n_{t_0} \,[\mathrm{cm}^{-3}]$')
ax.grid(True, which='both', axis='both', linestyle='--', alpha=0.6)
leg = ax.legend(framealpha=0.0, ncol=2, fontsize=28)
pf.apply_custom_settings(ax, leg, log_scale_y=False)
plt.savefig(f"plots/{finaldir}_n_dot.pdf", bbox_inches='tight')

fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(tiempo_s[1:], neutrons_number_density_cm3 + cumulative_net_absorption_emission_neutrons_inv_s_inv_cm3, label=r'$n_{n}$')  
ax.plot(tiempo_s[1:], protons_number_density_cm3 + cumulative_net_absorption_emission_protons_inv_s_inv_cm3, label=r'$n_{p}$')
ax.set_xlabel(r'$t\,[\mathrm{s}]$')
ax.set_ylabel(r'$n\,[\mathrm{{cm}}^{{-3}}]$')
ax.grid(True, which='both', axis='both', linestyle='--', alpha=0.6)
leg = ax.legend(framealpha=0.0, ncol=2, fontsize=28)
pf.apply_custom_settings(ax, leg, log_scale_y=False)
# ax.set_xscale('log')
plt.savefig(f"plots/{finaldir}_n_dot.pdf", bbox_inches='tight')

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(tiempo_s, Ye_dot_inv_s)
ax.set_xlabel(r'$t\,[\mathrm{s}]$', fontsize=28)
ax.set_ylabel(r'$\dot{Y}_e\;[\mathrm{s}^{-1}]$', fontsize=28)
leg = ax.legend(framealpha=0.0, fontsize=28)
ax.grid(True, which='both', axis='both', linestyle='--', alpha=0.6)
pf.apply_custom_settings(ax, leg, log_scale_y=False)
plt.savefig(f"plots/{finaldir}_Ye_dot.pdf", bbox_inches='tight')
plt.show()
plt.close(fig)

# Plot for cumulative change in Ye
fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(tiempo_s[1:], cumulative_Ye_inv_s, color='tab:blue', linewidth=2, marker='', markersize=5)
ax.set_xlabel(r'$t$ [s]', fontsize=28)
ax.set_ylabel(r'$Y_e-Y_e^{t_0}$', fontsize=28)
ax.grid(True, which='both', axis='both', linestyle='--', alpha=0.6)
leg = ax.legend(framealpha=0.95, fontsize=28)
pf.apply_custom_settings(ax, leg, log_scale_y=False)
plt.tight_layout()
plt.savefig(f"plots/{finaldir}_delta_Ye_cumulative.pdf", bbox_inches='tight')
plt.show()
plt.close(fig)

# Plot for Ye evolution
fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(tiempo_s[1:], Ye + cumulative_Ye_inv_s, color='tab:green', linewidth=2, marker='', markersize=5)
ax.set_xlabel(r'$t$ [s]', fontsize=28)
ax.set_ylabel(r'$Y_e$', fontsize=28)
ax.grid(True, which='both', axis='both', linestyle='--', alpha=0.6)
leg = ax.legend(framealpha=0.95, fontsize=28)
pf.apply_custom_settings(ax, leg, log_scale_y=False)
plt.tight_layout()
plt.savefig(f"plots/{finaldir}_Ye_evolution.pdf", bbox_inches='tight')
plt.show()
plt.close(fig)